In [1]:
# Install the happytransformer library using pip
!pip install happytransformer

# Import necessary libraries and modules
import csv
from datasets import load_dataset
from happytransformer import TTSettings, TTTrainArgs, HappyTextToText
from IPython.display import clear_output

# Clear the output in the IPython environment
clear_output()


In [2]:
# Create an instance of the HappyTextToText class for text-to-text tasks
happy_tt = HappyTextToText("T5", "t5-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [3]:
# Load datasets for training and evaluation using the jfleg dataset from the datasets library
train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[:]')


Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [4]:
def generate_csv(csv_path, dataset):
    """
    Generate a CSV file with specified input and target columns from a given dataset.

    Parameters:
    - csv_path (str): The path to the CSV file to be created.
    - dataset (list): The dataset containing input sentences and their corrections.

    Returns:
    None
    """
    with open(csv_path, 'w', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.writer(csvfile)

        # Write the header row with column names
        writer.writerow(["input", "target"])

        # Iterate through each case in the dataset
        for case in dataset:
            # Adding the task's prefix to the input text
            input_text = "grammar: " + case["sentence"]

            # Iterate through each correction in the case
            for correction in case["corrections"]:
                # Check if both input_text and correction are non-empty
                if input_text and correction:
                    # Write a row to the CSV file with input and target values
                    writer.writerow([input_text, correction])


In [5]:
# Generate CSV files for training and evaluation datasets using the defined function
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)


In [6]:
args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,1.255800,1.045646
34,0.827200,0.650572
68,0.698800,0.574394
102,0.755100,0.532961
136,0.667000,0.509549
170,0.597500,0.502558
204,0.587900,0.494026
238,0.624800,0.485699
272,0.590100,0.481653


Step,Training Loss,Validation Loss
1,1.255800,1.045646
34,0.827200,0.650572
68,0.698800,0.574394
102,0.755100,0.532961
136,0.667000,0.509549
170,0.597500,0.502558
204,0.587900,0.494026
238,0.624800,0.485699
272,0.590100,0.481653
306,0.544600,0.483253


In [7]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [12]:
example_1 = "grammar: The boys plays in AAST playground"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

The boys play in AAST playground.
